# <font color = 'blue'> Self Attention Decoder

# <font color = 'blue'> Import Libraries
As always, let's import all the required modules and set the random seeds for reproducability.

In [88]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import torchtext 
from torchtext.datasets import Multi30k
from torchtext.vocab import vocab

import numpy as np
from collections import Counter, OrderedDict

import random
import pandas as pd
from pathlib import Path

In [89]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [90]:
torchtext.__version__, torch.__version__, torch.cuda.is_available()

('0.11.0', '1.10.0', True)

# <font color = 'blue'>  Preparing the Data



In [91]:
data_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/Data/NLP')
project_folder = Path('/home/harpreet/Insync/google_drive_harpreet/Research/NLP/pytorch-seq2seq')

We'll then create our tokenizers as before.

## <font color = 'blue'> Load tokenized data

In [92]:
df_train = pd.read_pickle(project_folder/'df_train_en_de.pickel')

In [93]:
df_train

,source_tokens,target_tokens,source_tokens_reverse
0,"[zwei, junge, weiße, männer, sind, i, m, freie...","[two, young, ,, white, males, are, outside, ne...","[., büsche, vieler, nähe, der, in, freien, m, ..."
1,"[mehrere, männer, mit, schutzhelmen, bedienen,...","[several, men, in, hard, hats, are, operating,...","[., antriebsradsystem, ein, bedienen, schutzhe..."
2,"[ein, kleines, mädchen, klettert, in, ein, spi...","[a, little, girl, climbing, into, a, wooden, p...","[., holz, aus, spielhaus, ein, in, klettert, m..."
3,"[ein, mann, in, einem, blauen, hemd, steht, au...","[a, man, in, a, blue, shirt, is, standing, on,...","[., fenster, ein, putzt, und, leiter, einer, a..."
4,"[zwei, männer, stehen, am, herd, und, bereiten...","[two, men, are, at, the, stove, preparing, foo...","[., zu, essen, bereiten, und, herd, am, stehen..."
...,...,...,...
28995,"[., wand, verschnörkelten, einer, hinter, schr...","[a, woman, behind, a, scrolled, wall, is, writ...","[eine, frau, schreibt, hinter, einer, verschnö..."
28996,"[., kletterwand, einer, an, übt, bergsteiger, ...","[a, rock, climber, practices, on, a, rock, cli...","[ein, bergsteiger, übt, an, einer, kletterwand..."
28997,"[., hauses, einem, vor, straße, einer, auf, ar...","[two, male, construction, workers, are, workin...","[zwei, bauarbeiter, arbeiten, auf, einer, stra..."
28998,"[., fassade, einer, vor, wagen, einem, mit, ju...","[an, elderly, man, sits, outside, a, storefron...","[ein, älterer, mann, sitzt, mit, einem, jungen..."


## <font color = 'blue'> Small subset of data

In [94]:
df_train_small= df_train[0:4]

In [95]:
df_train_small

,source_tokens,target_tokens,source_tokens_reverse
0,"[zwei, junge, weiße, männer, sind, i, m, freie...","[two, young, ,, white, males, are, outside, ne...","[., büsche, vieler, nähe, der, in, freien, m, ..."
1,"[mehrere, männer, mit, schutzhelmen, bedienen,...","[several, men, in, hard, hats, are, operating,...","[., antriebsradsystem, ein, bedienen, schutzhe..."
2,"[ein, kleines, mädchen, klettert, in, ein, spi...","[a, little, girl, climbing, into, a, wooden, p...","[., holz, aus, spielhaus, ein, in, klettert, m..."
3,"[ein, mann, in, einem, blauen, hemd, steht, au...","[a, man, in, a, blue, shirt, is, standing, on,...","[., fenster, ein, putzt, und, leiter, einer, a..."


In [96]:
df_train_small= df_train_small.drop(columns=['source_tokens_reverse'])

In [97]:
df_train_small

,source_tokens,target_tokens
0,"[zwei, junge, weiße, männer, sind, i, m, freie...","[two, young, ,, white, males, are, outside, ne..."
1,"[mehrere, männer, mit, schutzhelmen, bedienen,...","[several, men, in, hard, hats, are, operating,..."
2,"[ein, kleines, mädchen, klettert, in, ein, spi...","[a, little, girl, climbing, into, a, wooden, p..."
3,"[ein, mann, in, einem, blauen, hemd, steht, au...","[a, man, in, a, blue, shirt, is, standing, on,..."


## <font color = 'blue'> Build Vocab

In [98]:
def create_vocab(text, min_freq, specials):
    my_counter = Counter()
    for line in text:
       my_counter.update(line)
    my_vocab = vocab(my_counter, min_freq=min_freq)
    for i, special in enumerate(specials):
        my_vocab.insert_token(special, i)
    my_vocab.set_default_index(0)
    return my_vocab

Create source vocab, We will add four special tokens - ```['<unk>', '<BOS>', '<EOS>', '<PAD>']```

### <font color = 'blue'> Source Vocab

In [99]:
source_vocab = create_vocab(df_train_small['source_tokens'], 1, ['<unk>', '<BOS>', '<EOS>', '<PAD>'])

In [100]:
len(source_vocab)

41

In [101]:
pd.DataFrame(source_vocab.get_stoi().items(), columns=['tokens', 'index']).sort_values(by = ['index'])[0:10]

,tokens,index
28,<unk>,0
34,<BOS>,1
23,<EOS>,2
19,<PAD>,3
17,zwei,4
27,junge,5
14,weiße,6
12,männer,7
16,sind,8
7,i,9


In [102]:
# check index of unknown word - it should be zero
source_vocab['abracdabra']

0

### <font color = 'blue'> Target Vocab

In [103]:
target_vocab = create_vocab(df_train_small['target_tokens'], 1, ['<unk>', '<BOS>', '<EOS>', '<PAD>'])

In [104]:
len(target_vocab)

40

## <font color = 'blue'> Create Dataset and Dataloader

In [105]:
class EngGerman(Dataset):
    def __init__(self, X1, X2):
        self.X1 = X1
        self.X2 = X2
        
    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, indices):
        return (self.X1.iloc[indices] , self.X2.iloc[indices]) 

In [106]:
trainset = EngGerman(df_train_small['source_tokens'], df_train['target_tokens'])

In [107]:
trainset[0]

(['zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'i',
  'm',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.'],
 ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.'])

In [108]:
trainset[1]

(['mehrere',
  'männer',
  'mit',
  'schutzhelmen',
  'bedienen',
  'ein',
  'antriebsradsystem',
  '.'],
 ['several',
  'men',
  'in',
  'hard',
  'hats',
  'are',
  'operating',
  'a',
  'giant',
  'pulley',
  'system',
  '.'])

In [109]:
trainset[2]

(['ein',
  'kleines',
  'mädchen',
  'klettert',
  'in',
  'ein',
  'spielhaus',
  'aus',
  'holz',
  '.'],
 ['a', 'little', 'girl', 'climbing', 'into', 'a', 'wooden', 'playhouse', '.'])

In [110]:
trainset[3]

(['ein',
  'mann',
  'in',
  'einem',
  'blauen',
  'hemd',
  'steht',
  'auf',
  'einer',
  'leiter',
  'und',
  'putzt',
  'ein',
  'fenster',
  '.'],
 ['a',
  'man',
  'in',
  'a',
  'blue',
  'shirt',
  'is',
  'standing',
  'on',
  'a',
  'ladder',
  'cleaning',
  'a',
  'window',
  '.'])

<font color = 'green'> **Function to replace words woth their index. Alaso add tokens BOS and EOS for beginning and end of sentences**

In [111]:
def text_transform (my_vocab, text):
     text_numerical = [my_vocab[token] for token in text]
     return torch.tensor([my_vocab['<BOS>']] + text_numerical + [my_vocab['<EOS>']])
     #return list(my_vocab['<BOS>']) + text_numerical + list(my_vocab['<EOS>'])

In [112]:
text = trainset[0][1]
print(text)
text_transform(target_vocab, text)

['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  2])

In [113]:
text = trainset[1][1]
print(text)
text_transform(target_vocab, text)

['several', 'men', 'in', 'hard', 'hats', 'are', 'operating', 'a', 'giant', 'pulley', 'system', '.']


tensor([ 1, 15, 16, 17, 18, 19,  9, 20, 21, 22, 23, 24, 14,  2])

<font color = 'green'> Create a function that will be use by dataloaders to group obsevations. We will first use transform function to add eos and bos tokens and replace words with indexes. Finally we will add pad tokens for smaller sentences in a batch.

In [114]:
def collate_batch(batch):
   source_list, target_list = [], []
   for (source_text, target_text) in batch:
        source_transform = text_transform(source_vocab, source_text)
        source_list.append(source_transform)
        target_transform =text_transform(target_vocab, target_text)
        target_list.append(target_transform)
        
   source_pad = pad_sequence(source_list, padding_value=3.0, batch_first = True)
   target_pad = pad_sequence(target_list, padding_value=3.0, batch_first = True)
   #print(source_list)
   return (source_pad, target_pad)

In [115]:
batch_size = 2

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                              collate_fn=collate_batch)

In [116]:
torch.manual_seed(456)
for i, (source, target) in enumerate(train_loader):
   
  print('batch number:' ,i)
  print('source')  
  print(source)
  print('target')  
  print(target)

batch number: 0
source
tensor([[ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  2,  3],
        [ 1, 22, 30, 12, 31, 32, 33, 34, 35, 36, 37, 38, 39, 22, 40, 17,  2]])
target
tensor([[ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  2,  3,  3,  3,  3],
        [ 1, 21, 31, 17, 21, 32, 33, 34, 35, 36, 21, 37, 38, 21, 39, 14,  2]])
batch number: 1
source
tensor([[ 1, 22, 24, 25, 26, 12, 22, 27, 28, 29, 17,  2],
        [ 1, 18,  7, 19, 20, 21, 22, 23, 17,  2,  3,  3]])
target
tensor([[ 1, 21, 25, 26, 27, 28, 21, 29, 30, 14,  2,  3,  3,  3],
        [ 1, 15, 16, 17, 18, 19,  9, 20, 21, 22, 23, 24, 14,  2]])


In [117]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
device

device(type='cuda')

In [118]:
src = source.clone()
print(src)
print(src.shape)

tensor([[ 1, 22, 24, 25, 26, 12, 22, 27, 28, 29, 17,  2],
        [ 1, 18,  7, 19, 20, 21, 22, 23, 17,  2,  3,  3]])
torch.Size([2, 12])


In [119]:
trg= target.clone
print(src)
print(src.shape)

tensor([[ 1, 22, 24, 25, 26, 12, 22, 27, 28, 29, 17,  2],
        [ 1, 18,  7, 19, 20, 21, 22, 23, 17,  2,  3,  3]])
torch.Size([2, 12])


# <font color = 'blue'> Decoder

## <font color = 'blue'>  **Embeddings**

In [120]:
trg = target.clone().to(device)

In [121]:
trg

tensor([[ 1, 21, 25, 26, 27, 28, 21, 29, 30, 14,  2,  3,  3,  3],
        [ 1, 15, 16, 17, 18, 19,  9, 20, 21, 22, 23, 24, 14,  2]],
       device='cuda:0')

In [122]:
hid_dim = 8
torch.manual_seed(0)
token_embedding_layer = nn.Embedding(len(target_vocab), hid_dim).to(device)

In [123]:
trg_embedding = token_embedding_layer(trg)

In [124]:
scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

In [125]:
trg_embedding_scaled = trg_embedding * scale

In [126]:
trg_embedding_scaled.shape
#[Batch_size, seq_len, hid_dim]

torch.Size([2, 14, 8])

In [127]:
position = torch.arange(trg.shape[1]).view(1,-1).repeat(trg.shape[0], 1).to(device)

In [128]:
position

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]],
       device='cuda:0')

In [129]:
max_length = 20
position_embedding_layer_decoder = nn.Embedding(max_length, hid_dim).to(device)

In [130]:
position_embedding = position_embedding_layer_decoder(position)

In [131]:
position_embedding.shape
#[Batch_size, seq_len, hid_dim]

torch.Size([2, 14, 8])

In [132]:
decoder_input_dropout = nn.Dropout(p=0.1)

In [133]:
decoder_input_after_dropout = decoder_input_dropout(position_embedding + trg_embedding_scaled)

In [134]:
decoder_input_after_dropout.shape

torch.Size([2, 14, 8])

## <font color = 'blue'> Decoder Layer

### <font color = 'blue'> Self Attention- Using nn.MultiheadAttention

<font color = 'green'> Now we will use ```torch.nn.MultiheadAttention``` from pytorch. The limittaions is that the hid_dim = out_hid_dim. Threfore hid_dim must be divisible by num_heads. In encoder hid_dim = 8, out_hid_dim =9 and num_heads =3. To keep things simpler for decoder, we will use hid_dim = out_hid_dim = 8. We will use num_heads = 2.

In [184]:
decoder_self_attnetion_layer = torch.nn.MultiheadAttention(embed_dim=hid_dim, num_heads=2, 
                                                        dropout=0.0, 
                                                        bias=True, add_bias_kv=False, 
                                                        add_zero_attn=False, kdim=None, 
                                                        vdim=None, batch_first=True, 
                                                        device=device, dtype=None)

In [185]:
for name, param in decoder_self_attnetion_layer.named_parameters():
    print(name, param)

in_proj_weight Parameter containing:
tensor([[ 1.8499e-01, -3.8478e-01,  7.1057e-02, -2.0518e-01, -3.1614e-02,
          1.0148e-01, -2.9991e-01, -2.7026e-01],
        [ 3.1315e-01, -2.5496e-01, -1.2828e-03, -3.8325e-01, -2.7335e-01,
          3.9826e-01, -2.6683e-01, -1.3738e-01],
        [-2.4088e-01, -2.8073e-01, -2.4275e-01, -3.1153e-01,  3.3613e-01,
          4.3101e-01, -1.8319e-01,  5.1078e-03],
        [-3.6488e-01, -2.4598e-02, -2.3206e-01,  2.5607e-01,  3.7520e-01,
          4.0791e-01,  8.6049e-03, -1.2862e-01],
        [-2.8392e-01,  1.9113e-01,  4.0404e-01, -2.3042e-01,  2.9782e-01,
          1.0228e-01,  1.5998e-01,  1.0465e-01],
        [ 2.0145e-01,  2.2270e-01,  1.2507e-01,  4.0968e-01, -2.3089e-01,
          3.3614e-01,  4.0727e-01,  2.7571e-01],
        [ 3.2707e-01, -3.5637e-01,  2.2451e-01, -3.0476e-01, -1.3932e-01,
         -2.4374e-01,  3.4372e-01,  9.7149e-02],
        [-2.5297e-01, -2.0734e-02,  3.1794e-01,  1.0522e-01,  6.3835e-02,
         -2.3669e-01,  7.188

<font size = 3, color = 'green'>**As we can see that this layers has two weight metrices. It combines $W^q, W^k and W^v$ into one matrix in_proj and $W^o$ is out_proj matrix.**

In [186]:
for name, param in decoder_self_attnetion_layer.named_parameters():
    print(name, param.shape)

in_proj_weight torch.Size([24, 8])
in_proj_bias torch.Size([24])
out_proj.weight torch.Size([8, 8])
out_proj.bias torch.Size([8])


In [187]:
trg

tensor([[ 1, 21, 25, 26, 27, 28, 21, 29, 30, 14,  2,  3,  3,  3],
        [ 1, 15, 16, 17, 18, 19,  9, 20, 21, 22, 23, 24, 14,  2]],
       device='cuda:0')

#### <font color ='blue'> Target Pad Mask

In [188]:
TRG_PAD_IDX = target_vocab['<PAD>']
TRG_PAD_IDX

3

<font color = 'green'> The layer apply mask where mask values are True. In manual implementation we applied mask where values are False. Hence now we will use ```trg_mask = (trg==TRG_PAD_IDX)``` instead of ```trg_mask = (trg!=TRG_PAD_IDX)```

In [189]:
trg_pad_mask = (trg==TRG_PAD_IDX)


In [190]:
trg_pad_mask.shape

torch.Size([2, 14])

#### <font color ='blue'> Target Attention Mask

In [191]:
trg_att_mask =~ torch.tril(torch.ones((trg.shape[1], trg.shape[1]), device = device)).bool()

In [192]:
trg_att_mask 

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False, False, False, False,

In [193]:
decoder_contextulaized_embeddings, decoder_self_attention_probs = decoder_self_attnetion_layer(
                                                     query=decoder_input_after_dropout, 
                                                     key= decoder_input_after_dropout,
                                                     value= decoder_input_after_dropout, 
                                                     key_padding_mask=trg_pad_mask, 
                                                     need_weights=True, 
                                                     attn_mask=trg_att_mask )

In [194]:
decoder_contextulaized_embeddings.shape

torch.Size([2, 14, 8])

In [195]:
decoder_contextulaized_embeddings[0, 0, :]
#[batch_size, query_len, hid_dim]

tensor([ 0.3303,  1.7884, -0.1772,  0.2327,  0.0448,  0.2538,  1.1147,  0.5120],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [196]:
decoder_self_attention_probs.shape
#[batch_size, query_len, key_len]

torch.Size([2, 14, 14])

In [197]:
decoder_self_attention_probs[0,0,:]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [198]:
decoder_self_attention_probs[0,13,:]

tensor([0.2371, 0.0075, 0.0228, 0.0816, 0.0046, 0.4548, 0.0119, 0.0390, 0.0058,
        0.0608, 0.0739, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [199]:
decoder_self_attention_probs[1,13,:]

tensor([6.2274e-01, 3.9103e-05, 6.3046e-03, 4.8590e-02, 2.7189e-02, 9.2773e-03,
        3.4019e-04, 1.6828e-02, 2.4630e-04, 1.0557e-01, 6.4785e-06, 5.2949e-02,
        1.0976e-01, 1.6127e-04], device='cuda:0', grad_fn=<SliceBackward0>)

### <font color = 'blue'> Method 1 - Self Attention- Without Using nn.MultiheadAttention

#### <font color = 'blue'> Manually initialize Weights and biases 

<font color = 'green'> To compare results, we will use the weights and biases from nn.MultiHeadAttention layer we used earlier.

In [200]:
out_hid_dim = 8
hid_dim = 8
fc_q = nn.Linear(hid_dim, out_hid_dim).to(device)
fc_k = nn.Linear(hid_dim, out_hid_dim).to(device)
fc_v = nn.Linear(hid_dim, out_hid_dim).to(device)
fc_o = nn.Linear(out_hid_dim, hid_dim).to(device)

In [201]:
fc_q.weight.data = decoder_self_attnetion_layer.in_proj_weight.data[0:8,:]
fc_q.bias.data = decoder_self_attnetion_layer.in_proj_bias.data[0:8]
print(fc_q.weight.data)
print(fc_q.bias.data)

tensor([[ 0.1850, -0.3848,  0.0711, -0.2052, -0.0316,  0.1015, -0.2999, -0.2703],
        [ 0.3132, -0.2550, -0.0013, -0.3832, -0.2734,  0.3983, -0.2668, -0.1374],
        [-0.2409, -0.2807, -0.2427, -0.3115,  0.3361,  0.4310, -0.1832,  0.0051],
        [-0.3649, -0.0246, -0.2321,  0.2561,  0.3752,  0.4079,  0.0086, -0.1286],
        [-0.2839,  0.1911,  0.4040, -0.2304,  0.2978,  0.1023,  0.1600,  0.1046],
        [ 0.2014,  0.2227,  0.1251,  0.4097, -0.2309,  0.3361,  0.4073,  0.2757],
        [ 0.3271, -0.3564,  0.2245, -0.3048, -0.1393, -0.2437,  0.3437,  0.0971],
        [-0.2530, -0.0207,  0.3179,  0.1052,  0.0638, -0.2367,  0.0719, -0.2264]],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [202]:
fc_k.weight.data = decoder_self_attnetion_layer.in_proj_weight.data[8:16,:]
fc_k.bias.data = decoder_self_attnetion_layer.in_proj_bias.data[8:16]
print(fc_k.weight.data)
print(fc_k.bias.data)

tensor([[ 0.2985,  0.3830,  0.3664,  0.3091, -0.2238, -0.3481,  0.3631,  0.2456],
        [-0.2906,  0.3379,  0.2518, -0.4016, -0.2576, -0.0500, -0.2887,  0.3279],
        [-0.1261,  0.3562,  0.4274, -0.3894, -0.2342, -0.0504,  0.3014,  0.0894],
        [-0.2343,  0.2767,  0.3893, -0.1903, -0.2182,  0.1408,  0.4273,  0.1500],
        [ 0.3249, -0.2130,  0.2843,  0.1670,  0.1609,  0.3828,  0.2209,  0.2019],
        [ 0.0038, -0.4129,  0.1820,  0.3375, -0.2741, -0.3686, -0.1665,  0.2397],
        [-0.4170,  0.0592, -0.2128,  0.1548, -0.3602, -0.2979,  0.2376,  0.3074],
        [ 0.3520, -0.0862, -0.1731,  0.3853,  0.1573,  0.0497, -0.2033,  0.0614]],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [203]:
fc_v.weight.data = decoder_self_attnetion_layer.in_proj_weight.data[16:,:]
fc_v.bias.data = decoder_self_attnetion_layer.in_proj_bias.data[16:]
print(fc_v.weight.data)
print(fc_v.bias.data)

tensor([[-2.4973e-01, -2.7177e-01, -5.6458e-02, -3.1782e-01, -2.1887e-01,
          1.3040e-01,  2.0355e-01,  1.5309e-01],
        [ 1.3054e-01,  1.6833e-01,  9.0505e-02, -1.8885e-01,  2.7652e-01,
         -7.3397e-02, -1.6197e-02,  1.4630e-01],
        [ 3.4320e-02,  2.2199e-01, -2.9434e-01, -1.0952e-01, -2.8894e-01,
          2.1007e-01, -9.3049e-02,  1.6416e-01],
        [ 2.4621e-02, -1.4539e-01, -3.7022e-02,  2.9981e-01, -1.3867e-01,
          2.1805e-01,  2.9434e-01, -1.2928e-02],
        [-3.8655e-01,  2.8785e-01,  2.5782e-01,  1.0498e-01, -2.7025e-01,
          2.8540e-04, -5.7410e-02, -3.2042e-01],
        [-2.3444e-01, -3.4577e-01,  1.9097e-01, -3.5385e-01, -2.7836e-01,
         -3.9557e-01, -1.0862e-01,  2.9519e-01],
        [ 5.0504e-02, -2.4758e-01,  2.5177e-01, -5.4291e-02, -6.0885e-02,
          1.8232e-01, -4.1251e-01, -1.7729e-02],
        [-4.0399e-01, -3.8897e-01, -2.8631e-01, -1.6947e-01, -8.0711e-02,
         -8.0041e-02, -7.5626e-02, -1.0621e-01]], device='cuda:0'

In [204]:
fc_o.weight.data = decoder_self_attnetion_layer.out_proj.weight.data
fc_o.bias.data = decoder_self_attnetion_layer.out_proj.bias.data
print(fc_o.weight.data)
print(fc_o.bias.data)

tensor([[ 0.3073,  0.2268, -0.3419, -0.2200,  0.3264, -0.3227, -0.0106,  0.1992],
        [-0.0231, -0.1152, -0.2776,  0.2452, -0.1651, -0.0271, -0.1799,  0.1165],
        [ 0.2908, -0.1879,  0.2437, -0.1677, -0.3395, -0.1987, -0.1430, -0.0004],
        [ 0.2092, -0.1042,  0.0999,  0.2766,  0.3155, -0.1570, -0.1468, -0.1715],
        [ 0.0373,  0.3285,  0.0616,  0.0057,  0.0457, -0.2534, -0.2507,  0.1994],
        [ 0.2037,  0.2375, -0.2708, -0.1696,  0.1607, -0.2345,  0.1412, -0.0931],
        [ 0.0733, -0.1727, -0.0976,  0.0093, -0.2668, -0.2439, -0.1830,  0.2035],
        [ 0.0248, -0.0544,  0.0215, -0.0652, -0.3079, -0.0412, -0.3170,  0.3218]],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [205]:
input_decoder_self = decoder_input_after_dropout.clone()
input_decoder_self.shape

torch.Size([2, 14, 8])

#### <font color = 'blue'> **Generate queries, keys and values**

In [206]:
Q = fc_q(input_decoder_self )
K = fc_k(input_decoder_self )
V = fc_v(input_decoder_self )

In [207]:
n_heads = 2
head_dim = out_hid_dim // n_heads
print(head_dim)

4


In [208]:
assert out_hid_dim % n_heads == 0

In [209]:
Q = Q.view(batch_size, -1, n_heads, head_dim)
K = K.view(batch_size, -1, n_heads, head_dim)
V = V.view(batch_size, -1, n_heads, head_dim)

In [210]:
Q.shape
#[batch_size, query_len, n_heads, head_dim]

torch.Size([2, 14, 2, 4])

In [211]:
Q = Q.permute(0, 2, 1, 3)
K = K.permute(0, 2, 1, 3)
V = V.permute(0, 2, 1, 3)

In [212]:
Q.shape
#[batch_size,num_heads, query_len, head_dim ]

torch.Size([2, 2, 14, 4])

In [213]:
K.shape
#[batch_size,num_heads, key_len, head_dim ]

torch.Size([2, 2, 14, 4])

#### <font color = 'blue'> **Scaled dot product of Queries and Keys**</font>

In [214]:
scale = torch.sqrt(torch.FloatTensor([head_dim])).to(device)

In [215]:
energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / scale

In [216]:
energy.shape
#[batch_size, num_heads, query_len, key_len]

torch.Size([2, 2, 14, 14])

#### <font color = 'blue'> **Apply mask to output of Q, K dot product**</font><br>
<font color = 'green'>**We do not want tokens to pay attention to pad tokens**</font>

In [217]:
mask_input = target.clone()
mask_input

tensor([[ 1, 21, 25, 26, 27, 28, 21, 29, 30, 14,  2,  3,  3,  3],
        [ 1, 15, 16, 17, 18, 19,  9, 20, 21, 22, 23, 24, 14,  2]])

In [218]:
TRG_PAD_IDX = target_vocab['<PAD>']
TRG_PAD_IDX

3

In [219]:
trg_pad_mask = (mask_input!= TRG_PAD_IDX )
trg_pad_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True]])

In [220]:
trg_pad_mask.shape

torch.Size([2, 14])

In [1]:
trg_pad_mask = trg_pad_mask.unsqueeze(1).unsqueeze(2)
#batch, num_head, query_len, key_len
# same mask will be applied for alll the heads

NameError: name 'trg_pad_mask' is not defined

In [222]:
trg_pad_mask.shape

torch.Size([2, 1, 1, 14])

In [223]:
trg_pad_mask= trg_pad_mask.to(device)

In [224]:
trg_att_mask1 = torch.tril(torch.ones(trg.shape[1], trg.shape[1])).bool().to(device)

In [225]:
trg_att_mask1 

tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,

In [226]:
trg_att_mask1.shape

torch.Size([14, 14])

In [227]:
trg_mask1 = trg_pad_mask & trg_att_mask1

In [228]:
trg_mask1.shape

torch.Size([2, 1, 14, 14])

In [229]:
trg_mask1

tensor([[[[ True, False, False, False, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True, False, False, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True, False, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True,  True, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True,  True,  True, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
           False, False, False, False],
          [ True,  True,  True, 

In [230]:
energy_masked = energy.masked_fill(trg_mask1 == 0, -1e10)

In [231]:
energy_masked.shape

torch.Size([2, 2, 14, 14])

#### <font color = 'blue'>**Apply softmax to convert QV dot product to probabilities**</font><br>

In [237]:
attention_prob = torch.softmax(energy_masked, dim = -1)                 
#attention_prob = [batch size, n heads, query len, key len]

In [238]:
print(attention_prob [1,1,1,12].data, attention_prob [1,1,1,13].data)
#[batch_size, num_heads, query_len, key_len]

tensor(0., device='cuda:0') tensor(0., device='cuda:0')


In [239]:
print(attention_prob[0, 0,1, :])

tensor([9.9969e-01, 3.0840e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0', grad_fn=<SliceBackward0>)


In [240]:
attention_prob[0, 0,1, :].sum()

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

#### <font color = 'blue'>**Apply dropout layer to attention probabilities**</font><br>

In [244]:
att_dec_dropout =  nn.Dropout(p=0.0)

In [245]:
attention_prob_after_dropout= att_dec_dropout(attention_prob)

In [246]:
print(attention_prob_after_dropout [0, 0,1, :])

tensor([9.9969e-01, 3.0840e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0', grad_fn=<SliceBackward0>)


In [247]:
attention_prob_after_dropout[0, 0,1, :].sum()

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

<font size = 3, color = 'red'>**Probs do not sum to 1, sometimes these are greater than one and sometimes these are less than one**</font>

In [248]:
attention_prob_after_dropout.shape

torch.Size([2, 2, 14, 14])

In [249]:
attention_prob_after_dropout_mean = torch.mean(attention_prob_after_dropout, dim = 1)

#### <font size = 4, color = 'blue'>**Self Attention Output**</font><br>

In [250]:
V.shape
# [batch_size, num_heads, value_len, head_dim]

torch.Size([2, 2, 14, 4])

In [251]:
attention_prob_after_dropout.shape
#[batch_size, num_heads, query_len, key_len]

torch.Size([2, 2, 14, 14])

In [252]:
decoder_contextulaized_embeddings_m1 = torch.matmul(attention_prob_after_dropout, V)
#[batch_size, num_heads, query_len, head_dim]

In [253]:
decoder_contextulaized_embeddings_m1.shape
# [batch_size, number_of_heads, query_len, head_dim]

torch.Size([2, 2, 14, 4])

In [254]:
decoder_contextulaized_embeddings_m1 = decoder_contextulaized_embeddings_m1.permute(0, 2, 1, 3)
# [batch_size, query_len, number_of_heads, head_dim]

In [255]:
decoder_contextulaized_embeddings_m1.shape

torch.Size([2, 14, 2, 4])

In [256]:
decoder_contextulaized_embeddings_m1 = decoder_contextulaized_embeddings_m1.reshape(batch_size, -1, out_hid_dim)

In [257]:
decoder_contextulaized_embeddings_m1.shape
#[batch_size, seq_len, out_hid_dim]

torch.Size([2, 14, 8])

<font size = 3, color = 'green'>We need to project the final values to have same shape as  input embedding. To accomplish this we will use fc_o linear layer we created earlier.

In [258]:
decoder_contextulaized_embeddings_m1 = fc_o(decoder_contextulaized_embeddings_m1)

In [259]:
decoder_contextulaized_embeddings_m1.shape
#[batch_size, seq_len, hid_dim]

torch.Size([2, 14, 8])

#### <font size = 4, color = 'blue'>**Compare results with PyTorch Layer**</font><br>

In [260]:
decoder_contextulaized_embeddings_m1[1,9,:]

tensor([ 0.1758,  1.1652, -1.1273,  1.3802, -0.4835, -0.0651, -0.1384, -0.5934],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [261]:
decoder_contextulaized_embeddings[1,9,:]

tensor([ 0.1758,  1.1652, -1.1273,  1.3802, -0.4835, -0.0651, -0.1384, -0.5934],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [262]:
decoder_self_attention_probs[0,5,:]

tensor([0.1110, 0.0380, 0.0864, 0.5372, 0.1614, 0.0661, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [263]:
attention_prob_after_dropout_mean[0,5,:]

tensor([0.1110, 0.0380, 0.0864, 0.5372, 0.1614, 0.0661, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SliceBackward0>)

### <font color = 'blue'> Method 2 - Self Attention- Without Using nn.MultiheadAttention

#### <font color = 'blue'> Manually initialize Weights and biases 

In [264]:
Wq = decoder_self_attnetion_layer.in_proj_weight.data[0:8,:]
bq = decoder_self_attnetion_layer.in_proj_bias.data[0:8]
print(Wq)
print(bq)

tensor([[ 0.1850, -0.3848,  0.0711, -0.2052, -0.0316,  0.1015, -0.2999, -0.2703],
        [ 0.3132, -0.2550, -0.0013, -0.3832, -0.2734,  0.3983, -0.2668, -0.1374],
        [-0.2409, -0.2807, -0.2427, -0.3115,  0.3361,  0.4310, -0.1832,  0.0051],
        [-0.3649, -0.0246, -0.2321,  0.2561,  0.3752,  0.4079,  0.0086, -0.1286],
        [-0.2839,  0.1911,  0.4040, -0.2304,  0.2978,  0.1023,  0.1600,  0.1046],
        [ 0.2014,  0.2227,  0.1251,  0.4097, -0.2309,  0.3361,  0.4073,  0.2757],
        [ 0.3271, -0.3564,  0.2245, -0.3048, -0.1393, -0.2437,  0.3437,  0.0971],
        [-0.2530, -0.0207,  0.3179,  0.1052,  0.0638, -0.2367,  0.0719, -0.2264]],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [265]:
Wk = decoder_self_attnetion_layer.in_proj_weight.data[8:16,:]
bk = decoder_self_attnetion_layer.in_proj_bias.data[8:16]
print(Wk)
print(bk)

tensor([[ 0.2985,  0.3830,  0.3664,  0.3091, -0.2238, -0.3481,  0.3631,  0.2456],
        [-0.2906,  0.3379,  0.2518, -0.4016, -0.2576, -0.0500, -0.2887,  0.3279],
        [-0.1261,  0.3562,  0.4274, -0.3894, -0.2342, -0.0504,  0.3014,  0.0894],
        [-0.2343,  0.2767,  0.3893, -0.1903, -0.2182,  0.1408,  0.4273,  0.1500],
        [ 0.3249, -0.2130,  0.2843,  0.1670,  0.1609,  0.3828,  0.2209,  0.2019],
        [ 0.0038, -0.4129,  0.1820,  0.3375, -0.2741, -0.3686, -0.1665,  0.2397],
        [-0.4170,  0.0592, -0.2128,  0.1548, -0.3602, -0.2979,  0.2376,  0.3074],
        [ 0.3520, -0.0862, -0.1731,  0.3853,  0.1573,  0.0497, -0.2033,  0.0614]],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [266]:
Wv = decoder_self_attnetion_layer.in_proj_weight.data[16:,:]
bv = decoder_self_attnetion_layer.in_proj_bias.data[16:]
print(Wv)
print(bv)

tensor([[-2.4973e-01, -2.7177e-01, -5.6458e-02, -3.1782e-01, -2.1887e-01,
          1.3040e-01,  2.0355e-01,  1.5309e-01],
        [ 1.3054e-01,  1.6833e-01,  9.0505e-02, -1.8885e-01,  2.7652e-01,
         -7.3397e-02, -1.6197e-02,  1.4630e-01],
        [ 3.4320e-02,  2.2199e-01, -2.9434e-01, -1.0952e-01, -2.8894e-01,
          2.1007e-01, -9.3049e-02,  1.6416e-01],
        [ 2.4621e-02, -1.4539e-01, -3.7022e-02,  2.9981e-01, -1.3867e-01,
          2.1805e-01,  2.9434e-01, -1.2928e-02],
        [-3.8655e-01,  2.8785e-01,  2.5782e-01,  1.0498e-01, -2.7025e-01,
          2.8540e-04, -5.7410e-02, -3.2042e-01],
        [-2.3444e-01, -3.4577e-01,  1.9097e-01, -3.5385e-01, -2.7836e-01,
         -3.9557e-01, -1.0862e-01,  2.9519e-01],
        [ 5.0504e-02, -2.4758e-01,  2.5177e-01, -5.4291e-02, -6.0885e-02,
          1.8232e-01, -4.1251e-01, -1.7729e-02],
        [-4.0399e-01, -3.8897e-01, -2.8631e-01, -1.6947e-01, -8.0711e-02,
         -8.0041e-02, -7.5626e-02, -1.0621e-01]], device='cuda:0'

In [267]:
Wo = decoder_self_attnetion_layer.out_proj.weight.data
bo = decoder_self_attnetion_layer.out_proj.bias.data
print(Wo)
print(bo)

tensor([[ 0.3073,  0.2268, -0.3419, -0.2200,  0.3264, -0.3227, -0.0106,  0.1992],
        [-0.0231, -0.1152, -0.2776,  0.2452, -0.1651, -0.0271, -0.1799,  0.1165],
        [ 0.2908, -0.1879,  0.2437, -0.1677, -0.3395, -0.1987, -0.1430, -0.0004],
        [ 0.2092, -0.1042,  0.0999,  0.2766,  0.3155, -0.1570, -0.1468, -0.1715],
        [ 0.0373,  0.3285,  0.0616,  0.0057,  0.0457, -0.2534, -0.2507,  0.1994],
        [ 0.2037,  0.2375, -0.2708, -0.1696,  0.1607, -0.2345,  0.1412, -0.0931],
        [ 0.0733, -0.1727, -0.0976,  0.0093, -0.2668, -0.2439, -0.1830,  0.2035],
        [ 0.0248, -0.0544,  0.0215, -0.0652, -0.3079, -0.0412, -0.3170,  0.3218]],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [268]:
input_decoder_self = decoder_input_after_dropout.clone()
input_decoder_self.shape

torch.Size([2, 14, 8])

#### <font color = 'blue'> **Generate queries, keys and values**

In [269]:
Q2 = input_decoder_self.matmul(Wq.T)  + bq
K2 = input_decoder_self.matmul(Wk.T)  + bk
V2 = input_decoder_self.matmul(Wv.T)  + bv

In [270]:
Q2.shape

torch.Size([2, 14, 8])

In [271]:
n_heads = 2
head_dim = out_hid_dim // n_heads
print(head_dim)

4


In [272]:
assert out_hid_dim % n_heads == 0

In [273]:
Q2 = Q2.view(batch_size, -1, n_heads, head_dim)
K2 = K2.view(batch_size, -1, n_heads, head_dim)
V2 = V2.view(batch_size, -1, n_heads, head_dim)

In [274]:
Q2.shape
#[batch_size, query_len, n_heads, head_dim]

torch.Size([2, 14, 2, 4])

In [275]:
Q2 = Q2.permute(0, 2, 1, 3)
K2 = K2.permute(0, 2, 1, 3)
V2 = V2.permute(0, 2, 1, 3)

In [276]:
Q2.shape
#[batch_size,num_heads, query_len, head_dim ]

torch.Size([2, 2, 14, 4])

In [277]:
K2.shape
#[batch_size,num_heads, key_len, head_dim ]

torch.Size([2, 2, 14, 4])

In [278]:
Q[1,1,5,:]

tensor([-3.4978,  0.3555,  0.9093, -2.7795], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [279]:
Q2[1,1,5,:]

tensor([-3.4978,  0.3555,  0.9093, -2.7795], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [280]:
K[1,1,5,:]

tensor([-0.4501,  0.8560,  1.7086,  0.9939], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [281]:
K2[1,1,5,:]

tensor([-0.4501,  0.8560,  1.7086,  0.9939], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [282]:
V[1,1,5,:]

tensor([-2.3665,  0.8371, -0.3593,  1.8726], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [283]:
V2[1,1,5,:]

tensor([-2.3665,  0.8371, -0.3593,  1.8726], device='cuda:0',
       grad_fn=<SliceBackward0>)

#### <font color = 'blue'> **Scaled dot product of Queries and Keys**</font>

In [284]:
scale = torch.sqrt(torch.FloatTensor([head_dim])).to(device)

In [285]:
energy2 = torch.matmul(Q2, K2.permute(0, 1, 3, 2)) / scale

In [286]:
energy2.shape
#[batch_size, num_heads, query_len, key_len]

torch.Size([2, 2, 14, 14])

In [287]:
energy[1,1,5,:]

tensor([-10.1447,   2.2105,   2.8566,  -1.8356,  -3.2271,   0.3348,   4.1132,
         -5.6259,   4.7959,  -1.5174,   6.9871,  -6.1813,  -5.2924,   2.2527],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [288]:
energy2[1,1,5,:]

tensor([-10.1447,   2.2105,   2.8566,  -1.8356,  -3.2271,   0.3348,   4.1132,
         -5.6259,   4.7959,  -1.5174,   6.9871,  -6.1813,  -5.2924,   2.2527],
       device='cuda:0', grad_fn=<SliceBackward0>)

#### <font color = 'blue'> **Apply mask to output of Q, K dot product**</font><br>
<font color = 'green'>**We do not want tokens to pay attention to pad tokens**</font>

In [289]:
mask_input2 = target.clone()
mask_input2

tensor([[ 1, 21, 25, 26, 27, 28, 21, 29, 30, 14,  2,  3,  3,  3],
        [ 1, 15, 16, 17, 18, 19,  9, 20, 21, 22, 23, 24, 14,  2]])

In [290]:
TRG_PAD_IDX = target_vocab['<PAD>']
TRG_PAD_IDX

3

In [291]:
trg_pad_mask2 = (mask_input2!= TRG_PAD_IDX )
trg_pad_mask2

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True]])

In [292]:
trg_pad_mask2.shape

torch.Size([2, 14])

In [293]:
trg_pad_mask2 = trg_pad_mask2.unsqueeze(1).unsqueeze(2)

In [294]:
trg_pad_mask2.shape

torch.Size([2, 1, 1, 14])

In [295]:
trg_pad_mask2 = trg_pad_mask2.to(device)

In [296]:
trg_att_mask2 = torch.tril(torch.ones(trg.shape[1], trg.shape[1])).bool().to(device)

In [297]:
trg_att_mask2 

tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,

In [298]:
trg_att_mask2.shape

torch.Size([14, 14])

In [299]:
trg_mask2 = trg_pad_mask2 & trg_att_mask2

In [300]:
trg_mask2.shape

torch.Size([2, 1, 14, 14])

In [301]:
trg_mask2

tensor([[[[ True, False, False, False, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True, False, False, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True, False, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True, False, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True, False, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True,  True, False, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True,  True,  True, False, False, False,
           False, False, False, False],
          [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
           False, False, False, False],
          [ True,  True,  True, 

In [302]:
energy_masked2 = energy2.masked_fill(trg_mask2 == 0, -1e10)

In [303]:
energy_masked2.shape

torch.Size([2, 2, 14, 14])

In [304]:
energy_masked[1,1,5,:]

tensor([-1.0145e+01,  2.2105e+00,  2.8566e+00, -1.8356e+00, -3.2271e+00,
         3.3482e-01, -1.0000e+10, -1.0000e+10, -1.0000e+10, -1.0000e+10,
        -1.0000e+10, -1.0000e+10, -1.0000e+10, -1.0000e+10], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [305]:
energy_masked2[1,1,5,:]

tensor([-1.0145e+01,  2.2105e+00,  2.8566e+00, -1.8356e+00, -3.2271e+00,
         3.3482e-01, -1.0000e+10, -1.0000e+10, -1.0000e+10, -1.0000e+10,
        -1.0000e+10, -1.0000e+10, -1.0000e+10, -1.0000e+10], device='cuda:0',
       grad_fn=<SliceBackward0>)

#### <font color = 'blue'>**Apply softmax to convert QV dot product to probabilities**</font><br>

In [306]:
attention_prob2 = torch.softmax(energy_masked2, dim = -1)                 
#attention_prob = [batch size, n heads, query len, key len]

In [307]:
print(attention_prob2[0,1,1,12].data, attention_prob2[0,1,1,13].data)
#[batch_size, num_heads, query_len, key_len]

tensor(0., device='cuda:0') tensor(0., device='cuda:0')


In [308]:
print(attention_prob2[0, 0,1, :])

tensor([9.9969e-01, 3.0840e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0', grad_fn=<SliceBackward0>)


In [309]:
attention_prob2[0, 0,1, :].sum()

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

#### <font color = 'blue'>**Apply dropout layer to attention probabilities**</font><br>
<font color = 'red'>**NOT UNDERSTOOD- why apply dropout here (probs will not sum to 1)**</font><br>
<font color = 'green'>**Quotes from paper --We apply dropout [33] to the output of each sub-layer, before it is added to the sub-layer input and normalized.**</font>


In [310]:
att_dec_dropout2 =  nn.Dropout(p=0.0)

In [311]:
attention_prob_after_dropout2 = att_dec_dropout2(attention_prob2)

In [312]:
print(attention_prob_after_dropout2[0, 0,1, :])

tensor([9.9969e-01, 3.0840e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0', grad_fn=<SliceBackward0>)


In [313]:
attention_prob_after_dropout2[0, 0,1, :].sum()

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

<font size = 3, color = 'red'>**Probs do not sum to 1, sometimes these are greater than one and sometimes these are less than one**</font>

In [314]:
attention_prob_after_dropout2.shape

torch.Size([2, 2, 14, 14])

In [315]:
attention_prob_after_dropout_mean2 = torch.mean(attention_prob_after_dropout2, dim = 1)

#### <font size = 4, color = 'blue'>**Self Attention Output**</font><br>

In [316]:
V2.shape
# [batch_size, num_heads, value_len, head_dim]

torch.Size([2, 2, 14, 4])

In [317]:
attention_prob_after_dropout2.shape
#[batch_size, num_heads, query_len, key_len]

torch.Size([2, 2, 14, 14])

In [318]:
decoder_contextulaized_embeddings_m2 = torch.matmul(attention_prob_after_dropout2, V2)
#[batch_size, num_heads, query_len, head_dim]

In [319]:
decoder_contextulaized_embeddings_m2.shape
# [batch_size, number_of_heads, query_len, head_dim]

torch.Size([2, 2, 14, 4])

In [320]:
decoder_contextulaized_embeddings_m2 = decoder_contextulaized_embeddings_m2.permute(0, 2, 1, 3)
# [batch_size, query_len, number_of_heads, head_dim]

In [321]:
decoder_contextulaized_embeddings_m2.shape

torch.Size([2, 14, 2, 4])

In [322]:
decoder_contextulaized_embeddings_m2 = decoder_contextulaized_embeddings_m2.reshape(batch_size, -1, out_hid_dim)

In [323]:
decoder_contextulaized_embeddings_m2.shape
#[batch_size, seq_len, out_hid_dim]

torch.Size([2, 14, 8])

<font size = 3, color = 'green'>We need to project the final values to have same shape as  input embedding. To accomplish this we will use fc_o linear layer we created earlier.

In [324]:
decoder_contextulaized_embeddings_m2 = decoder_contextulaized_embeddings_m2.matmul(Wo.T)  + bo

In [325]:
decoder_contextulaized_embeddings_m2.shape
#[batch_size, seq_len, hid_dim]

torch.Size([2, 14, 8])

#### <font size = 4, color = 'blue'>**Compare results with PyTorch Layer**</font><br>

In [326]:
decoder_contextulaized_embeddings_m2[0,9,:]

tensor([ 0.1856, -0.6724, -0.3055,  0.5599,  0.3984,  0.2749, -0.3462, -0.6872],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [327]:
decoder_contextulaized_embeddings[0,9,:]

tensor([ 0.1856, -0.6724, -0.3055,  0.5599,  0.3984,  0.2749, -0.3462, -0.6872],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [328]:
decoder_self_attention_probs[0,9,:]

tensor([0.0171, 0.0268, 0.0152, 0.0186, 0.1621, 0.6170, 0.0103, 0.0814, 0.0357,
        0.0158, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [329]:
attention_prob_after_dropout_mean2[0,9,:]

tensor([0.0171, 0.0268, 0.0152, 0.0186, 0.1621, 0.6170, 0.0103, 0.0814, 0.0357,
        0.0158, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SliceBackward0>)